# Réseaux convolutionnels : CNN

## Keras et Tensorflow

Pour installer Keras et Tensorflow sans GPU :

Pour installer la version GPU sous windows, cf https://medium.com/@raza.shahzad/setting-up-tensorflow-gpu-keras-in-conda-on-windows-10-75d4fd498198  
Sous Linux : http://deeplearning.lipingyang.org/2017/08/01/install-keras-with-tensorflow-backend/  
Sous MacOS (avec GPU Nvidia) : https://blog.wenhaolee.com/run-keras-on-mac-os-with-gpu/

## Initialisations

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import model_selection

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from sklearn import datasets

In [ ]:
from keras.datasets import mnist

from keras.models import Sequential, load_model

from keras.layers import Dense, Dropout, Flatten

from keras.layers.convolutional import Conv2D, MaxPooling2D

from keras.utils.np_utils import to_categorical

## Lecture des images

Pour une installation locale :
pip install opencv-python


In [ ]:
import cv2
import os
import glob

On utilise le dataset *Cat and Dog* : https://www.kaggle.com/tongpython/cat-and-dog

In [ ]:
# Affichage des fichiers dans le répertoire de données Kaggle
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

On parcourt le répertoire des images de chats, on lit chaque image, on la redimensionne et on l'ajoute au caractéristiques *X*. Pour les chats, on positionne la cible *y* à 0

In [ ]:
img_dir = "../input/cat-and-dog/training_set/training_set/cats" # Enter Directory of all images 
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
X=[]
y=[]
for f1 in files:
    img = cv2.imread(f1)
    img = cv2.resize(img, (100,100))
    X.append(np.array(img))
    y.append(0)
n_cats = len(X)

On affiche la première image :

In [ ]:
plt.imshow(X[0])
plt.title(y[0])

In [ ]:
np.array(X).shape

On affiche les 50 premières images de chats :

In [ ]:
plt.figure(figsize=(10,20))
for i in range(0,49) :
    plt.subplot(10,5,i+1)
    plt.axis('off')
    plt.imshow(X[i])
    plt.title('Label: %i' % y[i])

On lit de même les images de chiens, en positionnant la cible *y* à 1 :

In [ ]:
img_dir = "../input/cat-and-dog/training_set/training_set/dogs" # Enter Directory of all images 
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
for f1 in files:
    img = cv2.imread(f1)
    img = cv2.resize(img, (100,100))
    X.append(np.array(img))
    y.append(1)
n_dogs = len(X)-n_cats

In [ ]:
print(n_dogs)
print(n_cats)

In [ ]:
plt.figure(figsize=(10,20))
for i in range(0,49) :
    plt.subplot(10,5,i+1)
    plt.axis('off')
    plt.imshow(X[n_cats+i])
    plt.title('Label: %i' % y[n_cats+i])

On transforme les listes Python en tableau :

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
X.shape

In [ ]:
# Normalisation entre 0 et 1
X = X / 255
print(X[0][0])

On décompose en ensemble d'apprentissage et de validation :

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

## Une couche convolutionnelle

On va utiliser utiliser une couche convolutionnelle pour l'extraction des caractéristiques, et une couche dense pour la classification :

In [ ]:
# Réseau convolutionnel simple
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(100, 100, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1))

# Compilation du modèle
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])


On peut afficher la structure du modèle :

In [ ]:
model.summary()

In [ ]:
# Apprentissage
train = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=200, verbose=1)

In [ ]:
# Test
scores = model.evaluate(X_test, y_test, verbose=0)
print("Score : %.2f%%" % (scores[1]*100))

La variable *train* mémorise l'historique des scores sur l'ensemble d'apprentissage :

In [ ]:
print(train.history['accuracy'])

et sur l'ensemble de validation :

In [ ]:
print(train.history['val_accuracy'])

On définit une fonction pour afficher un graphique des scores :

In [ ]:
def plot_scores(train) :
    accuracy = train.history['accuracy']
    val_accuracy = train.history['val_accuracy']
    epochs = range(len(accuracy))
    plt.plot(epochs, accuracy, 'b', label='Score apprentissage')
    plt.plot(epochs, val_accuracy, 'r', label='Score validation')
    plt.title('Scores')
    plt.legend()
    plt.show()

In [ ]:
plot_scores(train)

In [ ]:
# Prediction
y_cnn = model.predict_classes(X_test)

On obtient un vecteur de probabilités :

In [ ]:
print(y_cnn.reshape(len(y_cnn)))

L'indice de la plus grande probabilité donne la classe prédite :

On peut afficher la matrice de confusion :

In [ ]:
cm = confusion_matrix(y_cnn,y_test)
print(cm)
plt.figure(figsize = (12,10))
sns.heatmap(cm, annot=True, cmap="coolwarm")

On affiche 50 images où l'algorithme s'est trompé :

In [ ]:
plt.figure(figsize=(15,25))
n_test = X_test.shape[0]
i=1
for j in range(len(X_test)) :
    if (y_cnn[j] != y_test[j]) & (i<50):
        plt.subplot(10,5,i)
        plt.axis('off')
        plt.imshow(X_test[j])
        pred_classe = y_cnn[j].argmax(axis=-1)
        plt.title('%d / %d' % (y_cnn[j], y_test[j]))
        i+=1

## Modèle CNN plus profond

On teste un modèle avec deux couches convolutionnelles :

In [ ]:
# Modèle CNN plus profond
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(100, 100, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

L'apprentissage peut être un peu long sans GPU ...

In [ ]:
# Apprentissage
train = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=200, verbose=1)

# Test
scores = model.evaluate(X_test, y_test, verbose=0)
print("Score : %.2f%%" % (scores[1]*100))

In [ ]:
plot_scores(train)

Le modèle entrainé peut être sauvegardé :

In [ ]:
model.save('mnist_cnn2.h5')

On peut ensuite utiliser le modèle sans recommencer l'entraînement :

In [ ]:
new_model = load_model('mnist_cnn2.h5')
new_model.summary()

In [ ]:
scores = new_model.evaluate(X_test, y_test, verbose=0)
print("Score : %.2f%%" % (scores[1]*100))

## Transfer learning

On va utiliser un modèle prédéfini dans Keras (VGG16) :

In [ ]:
from keras.applications import VGG16

On utilise les poids pré-entraînés sur ImageNet (un million d'images)
On "fige" le réseau VGG16, de manière à ne pas refaire l'entraînement sur le dataset particulier

In [ ]:
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(100,100,3))
vgg16.trainable = False

In [ ]:
vgg16.summary()

On ajoute des couches pour entraîner le modèle à partir du dataset, sans modifier les poids existants du VGG16 :

In [ ]:
model = Sequential()
model.add(vgg16)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [ ]:
train = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=200, verbose=1)

In [ ]:
# Test
scores = model.evaluate(X_test, y_test, verbose=0)
print("Score : %.2f%%" % (scores[1]*100))

In [ ]:
for i in range (len(vgg16.layers)):
    print (i,vgg16.layers[i])

On peut "dégeler" les dernières couches :

In [ ]:
for layer in vgg16.layers[15:]:
    layer.trainable=True
for layer in vgg16.layers[0:15]:
    layer.trainable=False

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
train = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=200, verbose=1)

In [ ]:
plot_scores(train)

In [ ]:
y_cnn = model.predict_classes(X_test)

In [ ]:
plt.figure(figsize=(15,25))
n_test = X_test.shape[0]
i=1
for j in range(len(X_test)) :
    if (y_cnn[j] != y_test[j]) & (i<50):
        plt.subplot(10,5,i)
        plt.axis('off')
        plt.imshow(X_test[j])
        pred_classe = y_cnn[j].argmax(axis=-1)
        plt.title('%d / %d' % (y_cnn[j], y_test[j]))
        i+=1

Il existe plusieurs autres modèles plus complexes :
https://keras.io/applications/

In [ ]:
from keras.applications import InceptionV3, ResNet50V2

## Exercices

Détection de pneumonie sur des radios :  
https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia  
  
Détection de cellules infectées par la malaria :  
https://www.kaggle.com/iarunava/cell-images-for-detecting-malaria